In [0]:
import os
import re
import sys
import urllib.request
import h5py

from keras import applications
from keras.models import Model
from keras.layers import Dropout, Dense, GlobalAveragePooling2D


from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.models import Sequential
from keras import optimizers

import numpy as np


def preprocess_input(x):
    x /= 255.
    x -= 0.5
    x *= 2.
    return x


class ModelBase:
    def __init__(self,
                 model_name=None,
                 batch_size=16,
                 verbose=0,
                 n_freeze_layers=0,
                 learning_rate=0.00001,
                 epochs=400,
                 optimizer='adam'
                 ):
        if model_name is None:
            script_name, script_ext = os.path.splitext(sys.argv[0])
            self.model_name = os.path.basename(script_name)
        else:
            self.model_name = model_name
        self.batch_size = batch_size
        self.verbose = verbose

        self.train_data_dir = "Train/TrainImages"
        self.validation_data_dir = "Validation/ValidationImages"
        self.test_data_dir = "Test/TestImages"
        self.img_width = 256
        self.img_height = 256
        self.img_channels = 3
        self.n_train_samples = 5830
        self.n_validation_samples = 2298
        self.n_test_samples = 3460
        self.n_labels = 29
        self.epochs = epochs
        self.n_freeze_layers = n_freeze_layers
        self.learning_rate = learning_rate
        self.imagenet_weights_url = None
        self.imagenet_use_id = False

    def load_model(self, model_weights=None):
        print('Creating model...')
        self._create()
        print('Loading weights from {}...'.format(model_weights))
        self.model.load_weights(model_weights)
        print('Compiling...')
        sgd = optimizers.SGD(lr=self.learning_rate, decay=1e-6, momentum=0.9, nesterov=True)
        adam = optimizers.Adam(lr=self.learning_rate)
        nadam = optimizers.Nadam(lr=self.learning_rate)
        self.model.compile(
            #optimizers.Adam(lr=self.learning_rate),
            #optimizers.Nadam(),
            optimizers.SGD(momentum=0.9),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        return self.model

    def train(self):
        print('Creating model...')
        self._create()

        self._freeze_top_layers()

        print('Loading weights...')
        self._load_pretrained_weights()

        print('Compiling...')
        self.model.compile(
            #optimizers.Adam(lr=self.learning_rate),
            #optimizers.Nadam(),
            optimizers.SGD(momentum=0.9),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

        #print(self.model.summary())

        # Data generators for the model
        train_gen = self._get_train_generator()
        validation_gen = self._get_validation_generator()

        print('Training model...')
        self.model.fit_generator(
            train_gen,
            steps_per_epoch=int(self.n_train_samples / self.batch_size),
            validation_data=validation_gen,
            validation_steps=int(self.n_validation_samples / self.batch_size),
            epochs=self.epochs,
            callbacks=self._get_callbacks(),
            verbose=self.verbose
        )

    def predict_validation(self, model_weights):
        img_paths = self._get_validation_image_paths()
        return self._predict(model_weights, img_paths)

    def predict_test(self, model_weights):
        img_paths = self._get_test_image_paths()
        return self._predict(model_weights, img_paths)

    @staticmethod
    def write_predictions(predictions, file_name='predictions.csv'):
        with open(file_name, 'w') as file:
            file.write('ID,Label')
            for index, value in enumerate(predictions):
                file.write('\n{0},{1}'.format(index + 1, value))

    @staticmethod
    def show_progress_bar(iteration, total, bar_length=50):
        percent = int(round((iteration / total) * 100))
        nb_bar_fill = int(round((bar_length * percent) / 100))
        bar_fill = '#' * nb_bar_fill
        bar_empty = ' ' * (bar_length - nb_bar_fill)
        sys.stdout.write("\r  [{0}] {1}%".format(str(bar_fill + bar_empty), percent))
        sys.stdout.flush()

    def _predict(self, model_weights, img_paths):
        self.load_model(model_weights)

        label_map = self._get_label_map()
        img_count = len(img_paths)
        y_predictions = np.zeros(img_count, dtype=np.int8)

        for i, (img_num, img_path) in enumerate(img_paths):
            ModelBase.show_progress_bar(i, img_count)
            img_data = self._load_image(img_path)
            if isinstance(self.model, Sequential):
                pred_index = self.model.predict_classes(img_data)[0]
            else:
                predictions = self.model.predict(img_data)
                pred_index = np.argmax(predictions, axis=1)[0]
            pred_label = label_map[pred_index]
            y_predictions[img_num-1] = pred_label

        return y_predictions

    def _get_label_map(self):
        # We need the ImageDataGenerator used to train the model
        # because it contains a mapping between classes and indices
        train_gen = self._get_train_generator()

        # Reverse keys and values so values becomes keys
        label_map = {v: int(k) for k, v in train_gen.class_indices.items()}

        return label_map

    def _load_image(self, image_path):
        img = image.load_img(image_path, target_size=(self.img_width, self.img_height))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        return preprocess_input(x)

    def _get_test_image_paths(self):
        final_list = []
        for img_name in os.listdir(self.test_data_dir):
            img_number = int(re.findall(r'\d+', img_name)[0])
            img_path = os.path.join(self.test_data_dir, img_name)
            final_list.append((img_number, img_path))
        return final_list

    def _get_validation_image_paths(self):
        final_list = []
        for sub_dir in os.listdir(self.validation_data_dir):
            sub_dir_path = os.path.join(self.validation_data_dir, sub_dir)
            for img_name in os.listdir(sub_dir_path):
                img_number = int(re.findall(r'\d+', img_name)[0])
                img_path = os.path.join(sub_dir_path, img_name)
                final_list.append((img_number, img_path))
        return final_list

    def _get_validation_generator(self):
        return ImageDataGenerator(
            preprocessing_function=preprocess_input
        ).flow_from_directory(
            self.validation_data_dir,
            target_size=(self.img_height, self.img_width),
            batch_size=self.batch_size,
            class_mode="categorical"
        )

    def _get_train_generator(self):
        return ImageDataGenerator(
            preprocessing_function=preprocess_input
        ).flow_from_directory(
            self.train_data_dir,
            target_size=(self.img_height, self.img_width),
            batch_size=self.batch_size,
            class_mode="categorical"
        )

    def _load_pretrained_weights(self):
        saved_weights_path = self._find_saved_weights()
        if saved_weights_path is not None:
            print('Loading saved weights from: {}'.format(saved_weights_path))
            self.model.load_weights(saved_weights_path)

        elif self.imagenet_weights_url is not None and len(self.imagenet_weights_url) > 0:
            print('Loading imagenet weights...')
            model_weights_path = 'saved_weights/{}'.format(os.path.basename(self.imagenet_weights_url))
            if os.path.isfile(model_weights_path):
                print('Model file already downloaded')
            else:
                # Download pre-trained weights
                print('Downloading {}...'.format(model_weights_path))
                urllib.request.urlretrieve(self.imagenet_weights_url, model_weights_path)
            self._load_weights_from_file(model_weights_path)
        else:
            print('No pre-trained weights loaded!')

    def _create(self):
        raise NotImplementedError('subclasses must override _create()')

    def _freeze_top_layers(self):
        if self.n_freeze_layers > 1:
            print("Freezing {} layers".format(self.n_freeze_layers))
            for layer in self.model.layers[:self.n_freeze_layers]:
                layer.trainable = False
            for layer in self.model.layers[self.n_freeze_layers:]:
                layer.trainable = True

    def _get_callbacks(self):
        # Define model checkpoint
        checkpoint = ModelCheckpoint(
            'saved_weights/%s-epoch{epoch:02d}-acc{acc:.2f}-loss{loss:.2f}'
            '-valacc{val_acc:.2f}-valloss{val_loss:.2f}.hdf5' % self.model_name,
            monitor='val_acc',
            save_best_only=False,
            save_weights_only=True,
            mode='auto',
            period=1,
            verbose=self.verbose
        )

        # Define early stopping
        early_stop = EarlyStopping(
            monitor='val_acc',
            min_delta=0,
            patience=10,
            mode='auto',
            verbose=self.verbose
        )
        
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-5)

        return [checkpoint, early_stop, reduce_lr]

    def _find_saved_weights(self, models_dir='./saved_weights/'):
        if not os.path.isdir(models_dir):
            return None

        list_of_files = sorted(os.listdir(models_dir))
        best_model = None
        best_acc = 0
        for f in list_of_files:
            if f.startswith(self.model_name):
                values = re.findall('val[^\d]*(\d+\.\d*)', f)
                acc = float(values[0])
                if acc > best_acc:
                    best_acc = acc
                    best_model = os.path.join(models_dir, f)
        return best_model

    def _load_weights_from_file(self, file_path):
        print('Loading weights from {}...'.format(file_path))

        layer_indices = {l.name: i for (i, l) in enumerate(self.model.layers)}

        # Load weights from the downloaded file
        with h5py.File(file_path) as model_weights_file:
            layer_names = model_weights_file.attrs['layer_names']
            for i, layer_name in enumerate(layer_names):
                level_0 = model_weights_file[layer_name]
                transferred_weights = []
                for k0 in level_0.keys():
                    level_1 = level_0[k0]
                    if hasattr(level_1, 'keys'):
                        for k1 in level_1.keys():
                            transferred_weights.append(level_1[k1][()])
                    else:
                        transferred_weights.append(level_0[k0][()])
                if self.imagenet_use_id:
                    layer_index = i
                else:
                    layer_key = layer_name.decode('UTF-8')
                    if layer_key not in layer_indices:
                        continue
                    layer_index = layer_indices[layer_key]
                self.model.layers[layer_index].set_weights(transferred_weights)
        print('Done loading weights')


Using TensorFlow backend.


In [0]:
class ModelInceptionV3Take2(ModelBase):
    def __init__(self, *args, **kwargs):
        ModelBase.__init__(self, *args, **kwargs)

    def _create(self):
        base_model = applications.InceptionV3(
            include_top=False,
            input_shape=(self.img_width, self.img_height, self.img_channels)
        )
        output_layer = base_model.output
        output_layer = GlobalAveragePooling2D(name='avg_pool')(output_layer)
        output_layer = Dense(256, activation='relu')(output_layer)
        output_layer = Dropout(0.5)(output_layer)
        output_layer = Dense(self.n_labels, activation='softmax', name='predictions')(output_layer)
        self.model = Model(inputs=base_model.input, outputs=output_layer)


In [0]:
m = ModelInceptionV3Take2(
    model_name='model-15-inception-v3-take3',
    n_freeze_layers=0,
    batch_size=64,
    verbose=1,
    epochs=300
)

In [0]:
m.train()

Creating model...
87916544/87910968 [==============================] - 5s 0us/step
Loading weights...
No pre-trained weights loaded!
Compiling...
Found 70208 images belonging to 29 classes.
Found 2298 images belonging to 29 classes.
Training model...
Epoch 1/300
91/91 [==============================] - 190s 2s/step - loss: 2.4741 - acc: 0.2886 - val_loss: 3.3572 - val_acc: 0.2379

Epoch 00001: saving model to saved_weights/model-15-inception-v3-take3-epoch01-acc0.29-loss2.47-valacc0.24-valloss3.36.hdf5
Epoch 2/300
41/91 [============>.................] - ETA: 1:23 - loss: 1.5905 - acc: 0.5225

91/91 [==============================] - 173s 2s/step - loss: 1.4171 - acc: 0.5628 - val_loss: 2.2122 - val_acc: 0.3795

Epoch 00002: saving model to saved_weights/model-15-inception-v3-take3-epoch02-acc0.56-loss1.42-valacc0.38-valloss2.21.hdf5
Epoch 3/300
91/91 [==============================] - 173s 2s/step - loss: 1.1295 - acc: 0.6429 - val_loss: 1.4330 - val_acc: 0.5629

Epoch 00003: saving model to saved_weights/model-15-inception-v3-take3-epoch03-acc0.64-loss1.13-valacc0.56-valloss1.43.hdf5
Epoch 4/300
91/91 [==============================] - 173s 2s/step - loss: 0.8998 - acc: 0.7141 - val_loss: 2.5088 - val_acc: 0.4402

Epoch 00004: saving model to saved_weights/model-15-inception-v3-take3-epoch04-acc0.71-loss0.90-valacc0.44-valloss2.51.hdf5
Epoch 5/300
 9/91 [=>............................] - ETA: 2:17 - loss: 0.8736 - acc: 0.7188

91/91 [==============================] - 173s 2s/step - loss: 0.8130 - acc: 0.7378 - val_loss: 1.4456 - val_acc: 0.6098

Epoch 00005: saving model to saved_weights/model-15-inception-v3-take3-epoch05-acc0.74-loss0.81-valacc0.61-valloss1.45.hdf5
Epoch 6/300
91/91 [==============================] - 173s 2s/step - loss: 0.6743 - acc: 0.7843 - val_loss: 0.9390 - val_acc: 0.7121

Epoch 00006: saving model to saved_weights/model-15-inception-v3-take3-epoch06-acc0.78-loss0.67-valacc0.71-valloss0.94.hdf5
Epoch 7/300
91/91 [==============================] - 173s 2s/step - loss: 0.6188 - acc: 0.8007 - val_loss: 1.4249 - val_acc: 0.6076

Epoch 00007: saving model to saved_weights/model-15-inception-v3-take3-epoch07-acc0.80-loss0.62-valacc0.61-valloss1.42.hdf5
Epoch 8/300
 6/91 [>.............................] - ETA: 2:22 - loss: 0.5771 - acc: 0.8281

91/91 [==============================] - 173s 2s/step - loss: 0.5765 - acc: 0.8070 - val_loss: 1.1056 - val_acc: 0.6661

Epoch 00008: saving model to saved_weights/model-15-inception-v3-take3-epoch08-acc0.81-loss0.58-valacc0.67-valloss1.11.hdf5
Epoch 9/300
91/91 [==============================] - 173s 2s/step - loss: 0.5236 - acc: 0.8343 - val_loss: 0.9495 - val_acc: 0.7210

Epoch 00009: saving model to saved_weights/model-15-inception-v3-take3-epoch09-acc0.83-loss0.52-valacc0.72-valloss0.95.hdf5
Epoch 10/300
91/91 [==============================] - 173s 2s/step - loss: 0.4661 - acc: 0.8486 - val_loss: 0.9884 - val_acc: 0.6987

Epoch 00010: saving model to saved_weights/model-15-inception-v3-take3-epoch10-acc0.85-loss0.47-valacc0.70-valloss0.99.hdf5
Epoch 11/300
 6/91 [>.............................] - ETA: 2:22 - loss: 0.4225 - acc: 0.8568

91/91 [==============================] - 173s 2s/step - loss: 0.4390 - acc: 0.8661 - val_loss: 1.0294 - val_acc: 0.7067

Epoch 00011: saving model to saved_weights/model-15-inception-v3-take3-epoch11-acc0.87-loss0.44-valacc0.71-valloss1.03.hdf5
Epoch 12/300
91/91 [==============================] - 173s 2s/step - loss: 0.3747 - acc: 0.8798 - val_loss: 0.4285 - val_acc: 0.8661

Epoch 00012: saving model to saved_weights/model-15-inception-v3-take3-epoch12-acc0.88-loss0.37-valacc0.87-valloss0.43.hdf5
Epoch 13/300
91/91 [==============================] - 173s 2s/step - loss: 0.2039 - acc: 0.9327 - val_loss: 0.3174 - val_acc: 0.8996

Epoch 00013: saving model to saved_weights/model-15-inception-v3-take3-epoch13-acc0.93-loss0.20-valacc0.90-valloss0.32.hdf5
Epoch 14/300
 6/91 [>.............................] - ETA: 2:21 - loss: 0.2329 - acc: 0.9271

...

91/91 [==============================] - 173s 2s/step - loss: 0.0311 - acc: 0.9921 - val_loss: 0.2391 - val_acc: 0.9348

Epoch 00041: saving model to saved_weights/model-15-inception-v3-take3-epoch41-acc0.99-loss0.03-valacc0.93-valloss0.24.hdf5
Epoch 42/300
91/91 [==============================] - 173s 2s/step - loss: 0.0311 - acc: 0.9911 - val_loss: 0.2439 - val_acc: 0.9313

Epoch 00042: saving model to saved_weights/model-15-inception-v3-take3-epoch42-acc0.99-loss0.03-valacc0.93-valloss0.24.hdf5
Epoch 43/300
91/91 [==============================] - 173s 2s/step - loss: 0.0295 - acc: 0.9923 - val_loss: 0.2370 - val_acc: 0.9353

Epoch 00043: saving model to saved_weights/model-15-inception-v3-take3-epoch43-acc0.99-loss0.03-valacc0.94-valloss0.24.hdf5
Epoch 44/300
 6/91 [>.............................] - ETA: 2:22 - loss: 0.0204 - acc: 0.9974

91/91 [==============================] - 173s 2s/step - loss: 0.0269 - acc: 0.9926 - val_loss: 0.2353 - val_acc: 0.9353

Epoch 00044: saving model to saved_weights/model-15-inception-v3-take3-epoch44-acc0.99-loss0.03-valacc0.94-valloss0.24.hdf5
Epoch 45/300
91/91 [==============================] - 173s 2s/step - loss: 0.0319 - acc: 0.9919 - val_loss: 0.2335 - val_acc: 0.9366

Epoch 00045: saving model to saved_weights/model-15-inception-v3-take3-epoch45-acc0.99-loss0.03-valacc0.94-valloss0.23.hdf5
Epoch 46/300
91/91 [==============================] - 173s 2s/step - loss: 0.0294 - acc: 0.9916 - val_loss: 0.2430 - val_acc: 0.9353

Epoch 00046: saving model to saved_weights/model-15-inception-v3-take3-epoch46-acc0.99-loss0.03-valacc0.94-valloss0.24.hdf5
Epoch 47/300
 6/91 [>.............................] - ETA: 2:22 - loss: 0.0590 - acc: 0.9844

91/91 [==============================] - 173s 2s/step - loss: 0.0317 - acc: 0.9906 - val_loss: 0.2473 - val_acc: 0.9330

Epoch 00047: saving model to saved_weights/model-15-inception-v3-take3-epoch47-acc0.99-loss0.03-valacc0.93-valloss0.25.hdf5
Epoch 48/300
91/91 [==============================] - 173s 2s/step - loss: 0.0279 - acc: 0.9914 - val_loss: 0.2393 - val_acc: 0.9344

Epoch 00048: saving model to saved_weights/model-15-inception-v3-take3-epoch48-acc0.99-loss0.03-valacc0.93-valloss0.24.hdf5
Epoch 49/300
91/91 [==============================] - 173s 2s/step - loss: 0.0256 - acc: 0.9933 - val_loss: 0.2398 - val_acc: 0.9366

Epoch 00049: saving model to saved_weights/model-15-inception-v3-take3-epoch49-acc0.99-loss0.03-valacc0.94-valloss0.24.hdf5
Epoch 50/300
 6/91 [>.............................] - ETA: 2:21 - loss: 0.0433 - acc: 0.9870

91/91 [==============================] - 173s 2s/step - loss: 0.0308 - acc: 0.9906 - val_loss: 0.2463 - val_acc: 0.9326

Epoch 00050: saving model to saved_weights/model-15-inception-v3-take3-epoch50-acc0.99-loss0.03-valacc0.93-valloss0.25.hdf5
Epoch 51/300
91/91 [==============================] - 173s 2s/step - loss: 0.0249 - acc: 0.9936 - val_loss: 0.2413 - val_acc: 0.9357

Epoch 00051: saving model to saved_weights/model-15-inception-v3-take3-epoch51-acc0.99-loss0.02-valacc0.94-valloss0.24.hdf5
Epoch 52/300
91/91 [==============================] - 173s 2s/step - loss: 0.0273 - acc: 0.9935 - val_loss: 0.2374 - val_acc: 0.9353

Epoch 00052: saving model to saved_weights/model-15-inception-v3-take3-epoch52-acc0.99-loss0.03-valacc0.94-valloss0.24.hdf5
Epoch 53/300
 6/91 [>.............................] - ETA: 2:22 - loss: 0.0309 - acc: 0.9922

91/91 [==============================] - 173s 2s/step - loss: 0.0286 - acc: 0.9924 - val_loss: 0.2394 - val_acc: 0.9362

Epoch 00053: saving model to saved_weights/model-15-inception-v3-take3-epoch53-acc0.99-loss0.03-valacc0.94-valloss0.24.hdf5
Epoch 54/300
91/91 [==============================] - 173s 2s/step - loss: 0.0272 - acc: 0.9935 - val_loss: 0.2373 - val_acc: 0.9362

Epoch 00054: saving model to saved_weights/model-15-inception-v3-take3-epoch54-acc0.99-loss0.03-valacc0.94-valloss0.24.hdf5
Epoch 55/300
91/91 [==============================] - 173s 2s/step - loss: 0.0218 - acc: 0.9950 - val_loss: 0.2382 - val_acc: 0.9362

Epoch 00055: saving model to saved_weights/model-15-inception-v3-take3-epoch55-acc1.00-loss0.02-valacc0.94-valloss0.24.hdf5
Epoch 00055: early stopping


In [0]:
pwd

'/content/cvml/project'

In [0]:
!git clone https://github.com/sheikhomar/cvml.git

Cloning into 'cvml'...
remote: Counting objects: 78328, done.
remote: Total 78328 (delta 0), reused 0 (delta 0), pack-reused 78328
Receiving objects: 100% (78328/78328), 476.18 MiB | 30.65 MiB/s, done.
Resolving deltas: 100% (736/736), done.
Checking out files: 100% (77993/77993), done.


In [0]:
ls -haltr saved_weights/

total 4.7G
-rw-r--r-- 1 root root    0 May  7 03:22 sentinel
drwxr-xr-x 9 root root 4.0K May  7 03:22 ../
-rw-r--r-- 1 root root  86M May  7 03:27 model-15-inception-v3-take3-epoch01-acc0.29-loss2.47-valacc0.24-valloss3.36.hdf5
-rw-r--r-- 1 root root  86M May  7 03:30 model-15-inception-v3-take3-epoch02-acc0.56-loss1.42-valacc0.38-valloss2.21.hdf5
-rw-r--r-- 1 root root  86M May  7 03:33 model-15-inception-v3-take3-epoch03-acc0.64-loss1.13-valacc0.56-valloss1.43.hdf5
-rw-r--r-- 1 root root  86M May  7 03:36 model-15-inception-v3-take3-epoch04-acc0.71-loss0.90-valacc0.44-valloss2.51.hdf5
-rw-r--r-- 1 root root  86M May  7 03:39 model-15-inception-v3-take3-epoch05-acc0.74-loss0.81-valacc0.61-valloss1.45.hdf5
-rw-r--r-- 1 root root  86M May  7 03:42 model-15-inception-v3-take3-epoch06-acc0.78-loss0.67-valacc0.71-valloss0.94.hdf5
-rw-r--r-- 1 root root  86M May  7 03:45 model-15-inception-v3-take3-epoch07-acc0.80-loss0.62-valacc0.61-valloss1.42.hdf5
-rw-r--r-- 1 root root  86M May  7 03:47

In [0]:
from google.colab import files
files.download('saved_weights/model-15-inception-v3-take3-epoch45-acc0.99-loss0.03-valacc0.94-valloss0.23.hdf5')


In [0]:

from google.colab import files
files.download('saved_weights/model-15-inception-v3-take3-epoch55-acc1.00-loss0.02-valacc0.94-valloss0.24.hdf5')

In [0]:
y_val_pred = m.predict_validation(model_weights='saved_weights/model-15-inception-v3-take3-epoch55-acc1.00-loss0.02-valacc0.94-valloss0.24.hdf5')

Creating model...
Loading weights from saved_weights/model-15-inception-v3-take3-epoch55-acc1.00-loss0.02-valacc0.94-valloss0.24.hdf5...
Compiling...
Found 70208 images belonging to 29 classes.
  [########                                          ] 17%

  [##################################################] 100%

In [0]:
from sklearn.metrics import classification_report
import pandas as pd


y_validation = pd.read_csv('Validation/valLbls.csv', header=None, names=['label'])['label']
print(classification_report(y_validation, y_val_pred))


             precision    recall  f1-score   support

          1       1.00      0.99      1.00       116
          2       0.86      0.84      0.85        96
          3       0.90      0.96      0.93        94
          4       0.99      0.99      0.99        92
          5       0.97      0.99      0.98        88
          6       0.96      0.83      0.89        92
          7       0.89      0.84      0.86        92
          8       0.83      0.89      0.86        88
          9       0.97      0.99      0.98        88
         10       0.96      0.91      0.94        82
         11       0.99      0.98      0.98        86
         12       0.93      0.96      0.94        80
         13       0.98      1.00      0.99        80
         14       0.89      0.96      0.92        82
         15       0.94      0.90      0.92        82
         16       0.94      0.99      0.96        82
         17       1.00      0.99      0.99        80
         18       0.97      0.97      0.97   

In [0]:
y_test_pred = m.predict_test(model_weights='saved_weights/model-15-inception-v3-take3-epoch55-acc1.00-loss0.02-valacc0.94-valloss0.24.hdf5')

Creating model...
Loading weights from saved_weights/model-15-inception-v3-take3-epoch55-acc1.00-loss0.02-valacc0.94-valloss0.24.hdf5...
Compiling...
Found 70208 images belonging to 29 classes.
  [######                                            ] 11%

  [##################################################] 100%

In [0]:
ModelBase.write_predictions(y_test_pred, file_name='test-pred-model-15-inception-v3-take3-epoch55-acc1.00-loss0.02-valacc0.94-valloss0.24.csv')

In [0]:
files.download('test-pred-model-15-inception-v3-take3-epoch55-acc1.00-loss0.02-valacc0.94-valloss0.24.csv')